In [1]:
!pip install torch-scatter -f https://data.pyg.org/whl/torch-2.2.0+cu121.html
!pip install torch-sparse  -f https://data.pyg.org/whl/torch-2.2.0+cu121.html
!pip install torch-geometric

!pip install --use-deprecated=legacy-resolver karateclub networkx numpy pandas matplotlib scikit-learn

!pip install torch torchvision torchaudio
!pip install torch-geometric \
    -f https://data.pyg.org/whl/torch-$(python -c "import torch; print(torch.__version__)").html


!pip install optuna
!pip install karateclub

Looking in links: https://data.pyg.org/whl/torch-2.2.0+cu121.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 55.2 MB/s eta 0:00:00
Looking in links: https://data.pyg.org/whl/torch-2.2.0+cu121.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 41.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 23.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.5/64.5 kB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 28.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.9/27.9 MB 19.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.3/153.3 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 77.3 MB/s eta 0:00:00
  Created wheel for karateclub: filename=karateclub-1.3.3-py3-none-any.whl size=101979 sha256=17c4976764feb585

In [67]:
import os, time, json
import numpy as np
import pandas as pd
import torch

from google.colab import drive
drive.mount('/content/drive')
from torch_geometric.datasets import TUDataset
from torch_geometric.loader import DataLoader
from torch_geometric.nn import GINConv, global_add_pool
import torch.nn as nn
import torch.nn.functional as F
import copy
import networkx as nx
from karateclub import Graph2Vec, NetLSD
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

# Paths
CLASS_DIR = "/content/drive/MyDrive/InformationSystems/Classification/embeddings"
BASE_DIR  = "/content/drive/MyDrive/InformationSystems/Stability"
RESULTS_DIR = f"{BASE_DIR}/results"
PLOTS_DIR   = f"{BASE_DIR}/plots"   # προαιρετικό, για plots stability curves

os.makedirs(BASE_DIR, exist_ok=True)
os.makedirs(RESULTS_DIR, exist_ok=True)
os.makedirs(PLOTS_DIR, exist_ok=True)

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", DEVICE)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Device: cpu


In [87]:
def load_dataset(dataset,root="/content/data"):
    ds = TUDataset(root=root, name=dataset)
    # ds[i] is a torch_geometric.data.Data with x, edge_index, y
    print(f"Loaded {dataset}:", len(ds), "graphs",
          "| num_classes:", ds.num_classes,
          "| num_node_features:", ds.num_node_features)
    return ds

In [69]:
def get_nonempty_graph_indices(dataset):
    """Return indices of graphs with at least 3 nodes. """
    idxs = []
    for i in range(len(dataset)):
      d = dataset[i]
      n = int(d.num_nodes) if d.num_nodes is not None else 0
      if n > 2:
        idxs.append(i)

    removed = len(dataset) - len(idxs)
    if removed > 0:
        print(f"[INFO] Filtered out {removed} ENZYMES graphs with 0 nodes for stability.")

    return idxs

In [70]:
def _to_undirected_edge_index(edge_index, num_nodes):
    # ensure undirected without duplicates
    # edge_index: [2, E]
    row, col = edge_index
    rev = torch.stack([col, row], dim=0)
    ei = torch.cat([edge_index, rev], dim=1)
    # remove duplicates
    ei = torch.unique(ei, dim=1)
    # remove self-loops (optional)
    mask = ei[0] != ei[1]
    return ei[:, mask]

def perturb_edges(data,
                  remove_pct: float = 0.10,
                  add_pct: float = 0.10,
                  seed: int = 42,
                  allow_self_loops: bool = False):
    """
    Randomly remove % of edges and add % new random edges.
    Returns a NEW Data object.
    """
    rng = np.random.default_rng(seed)
    d = copy.deepcopy(data)

    num_nodes = int(d.num_nodes)
    ei = d.edge_index.clone()

    # Make undirected (ENZYMES is undirected)
    ei = _to_undirected_edge_index(ei, num_nodes)

    # Convert edges to numpy pairs
    edges = ei.t().cpu().numpy()  # (E,2)
    E = edges.shape[0]

    # --- remove edges ---
    n_remove = int(remove_pct * E)
    if n_remove > 0 and E > 0:
        idx_remove = rng.choice(E, size=min(n_remove, E), replace=False)
        keep_mask = np.ones(E, dtype=bool)
        keep_mask[idx_remove] = False
        edges = edges[keep_mask]
    else:
        edges = edges

    # --- add edges ---
    edges_set = set(map(tuple, edges))
    n_add = int(add_pct * max(1, E))

    tries = 0
    max_tries = max(1000, 10 * n_add)
    added = 0

    while added < n_add and tries < max_tries:
        u = int(rng.integers(0, num_nodes))
        v = int(rng.integers(0, num_nodes))
        tries += 1

        if (not allow_self_loops) and (u == v):
            continue

        # undirected: store both directions
        if (u, v) in edges_set or (v, u) in edges_set:
            continue

        edges_set.add((u, v))
        edges_set.add((v, u))
        added += 1

    new_edges = np.array(list(edges_set), dtype=np.int64)
    new_ei = torch.from_numpy(new_edges).t().contiguous()

    d.edge_index = new_ei
    return d

In [71]:
def shuffle_node_attributes(data, seed: int = 42, mode: str = "permute_rows"):
    """
    Shuffle node attributes.
    mode:
      - "permute_rows": permute nodes (shuffle x rows)
      - "shuffle_columns": shuffle each feature column independently
    """
    rng = np.random.default_rng(seed)
    d = copy.deepcopy(data)

    if getattr(d, "x", None) is None:
        # ENZYMES έχει x, αλλά κρατάμε safe fallback
        return d

    x = d.x.clone().cpu().numpy()
    n = x.shape[0]

    if mode == "permute_rows":
        perm = rng.permutation(n)
        x = x[perm]
    elif mode == "shuffle_columns":
        for j in range(x.shape[1]):
            rng.shuffle(x[:, j])
    else:
        raise ValueError("Unknown mode: " + mode)

    d.x = torch.from_numpy(x).to(d.x.dtype)
    return d

In [72]:
def make_perturbed_graphs(dataset,
                          edge_remove_pct: float,
                          edge_add_pct: float,
                          attr_shuffle: bool,
                          seed: int,
                          indices: list | None = None):
    """Create perturbated graphs for the given indices (or all graphs if indices=None)."""
    if indices is None:
      indices = list (range(len(dataset)))

    perturbed = []
    for pos,i in enumerate(indices):
        g = dataset[i]
        g2 = perturb_edges(g, remove_pct=edge_remove_pct, add_pct=edge_add_pct, seed=seed + pos)
        if attr_shuffle:
            g2 = shuffle_node_attributes(g2, seed=seed + 10_000 + pos, mode="permute_rows")
        perturbed.append(g2)
    return perturbed

In [73]:
def load_embeddings_and_labels(method_name: str, experiment_num: str, base_dir: str = CLASS_DIR):
    exp_dir = os.path.join(base_dir, method_name, str(experiment_num))
    emb_path = os.path.join(exp_dir, "embeddings.npy")
    labels_path = os.path.join(exp_dir, "labels.npy")

    if not os.path.exists(emb_path):
        raise FileNotFoundError(f"Embeddings file not found: {emb_path}")
    if not os.path.exists(labels_path):
        raise FileNotFoundError(f"Labels file not found: {labels_path}")

    emb = np.load(emb_path)
    y = np.load(labels_path)
    return emb, y

In [74]:
def pyg_to_nx(data):
    G = nx.Graph()
    G.add_nodes_from(range(int(data.num_nodes)))

    ei = data.edge_index.cpu().numpy()
    edges = list(zip(ei[0], ei[1]))
    G.add_edges_from(edges)
    return G

In [75]:
def svm_accuracy(emb: np.ndarray, y: np.ndarray, C=10.0, gamma="scale", seed: int = 42):
    X_train, X_test, y_train, y_test = train_test_split(
        emb, y, test_size=0.2, random_state=seed, stratify=y # na do split sto classification
    )
    clf = make_pipeline(StandardScaler(), SVC(C=C, gamma=gamma, kernel="rbf"))
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    return float(accuracy_score(y_test, y_pred))

In [95]:
# -------------------------
# GIN (PyG) utilities
# -------------------------
class GINEncoderClassifier(nn.Module):
    """GIN encoder + graph-level pooling + classifier. Also returns graph embeddings."""
    def __init__(self, in_dim: int, hidden_dim: int, num_layers: int, num_classes: int, dropout: float = 0.5):
        super().__init__()
        self.dropout = dropout
        self.convs = nn.ModuleList()
        self.bns = nn.ModuleList()

        # First layer
        mlp = nn.Sequential(
            nn.Linear(in_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim)
        )
        self.convs.append(GINConv(mlp))
        self.bns.append(nn.BatchNorm1d(hidden_dim))

        # Remaining layers

        for _ in range(num_layers -1):
            mlp = nn.Sequential(
                nn.Linear(hidden_dim, hidden_dim),
                nn.ReLU(),
                nn.Linear(hidden_dim, hidden_dim)
            )

            self.convs.append(GINConv(mlp))
            self.bns.append(nn.BatchNorm1d(hidden_dim))

        self.classifier = nn.Linear(hidden_dim, num_classes)

    def forward(self, x, edge_index, batch):
      for conv, bn in zip(self.convs, self.bns):
            x = conv(x, edge_index)
            x = bn(x)
            x = F.relu(x)
            x = F.dropout(x, p=self.dropout, training=self.training)
      g = global_add_pool(x, batch)  # graph embedding
      out = self.classifier(g)
      return out, g


In [94]:

def _make_split_indices(n: int, seed: int = 42, test_size: float = 0.2):
    rng = np.random.default_rng(seed)
    idx = np.arange(n)
    rng.shuffle(idx)
    n_test = int(round(test_size * n))
    test_idx = idx[:n_test]
    train_idx = idx[n_test:]
    return train_idx, test_idx


def train_gin_model(model: nn.Module,
                    train_loader: DataLoader,
                    device: torch.device,
                    epochs: int = 80,
                    lr: float = 1e-3,
                    weight_decay: float = 5e-4):
    model.to(device)
    opt = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
    criterion = nn.CrossEntropyLoss()

    start = time.time()
    for _ in range(epochs):
        model.train()
        for batch in train_loader:
            batch = batch.to(device)
            opt.zero_grad()
            logits, _ = model(batch.x, batch.edge_index, batch.batch)
            loss = criterion(logits, batch.y)
            loss.backward()
            opt.step()

    return time.time() - start


def infer_gin_embeddings(model: nn.Module, loader: DataLoader, device: torch.device):
    model.eval()
    embs = []
    with torch.no_grad():
        for batch in loader:
            batch = batch.to(device)
            _, g = model(batch.x, batch.edge_index, batch.batch)
            embs.append(g.detach().cpu().numpy())
    return np.vstack(embs)


def gin_accuracy(model: nn.Module, loader: DataLoader, device: torch.device):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for batch in loader:
            batch = batch.to(device)
            logits, _ = model(batch.x, batch.edge_index, batch.batch)
            pred = logits.argmax(dim=1)
            correct += int((pred == batch.y).sum().item())
            total += int(batch.y.size(0))
    return float(correct / max(1, total))


In [79]:
def compute_embeddings_gin(graphs_pyg: list,
                           labels: np.ndarray,
                           hidden_dim: int,
                           num_layers: int,
                           dropout: float,
                           epochs: int,
                           batch_size: int,
                           lr: float,
                           weight_decay: float,
                           seed: int,
                           device: torch.device):
    """Train a GIN on the provided graphs and return graph embeddings for ALL graphs."""
    # Ensure graphs have node features
    for i, g in enumerate(graphs_pyg):
        if getattr(g, "x", None) is None:
            raise ValueError(f"Graph {i} has no node features (x). GIN requires node features.")

    # Attach labels to graphs (PyG expects tensor y)
    graphs = []
    for g, y in zip(graphs_pyg, labels):
        gg = copy.deepcopy(g)
        gg.y = torch.tensor(int(y), dtype=torch.long)
        graphs.append(gg)

    n = len(graphs)
    train_idx, test_idx = _make_split_indices(n, seed=seed, test_size=0.2)

    train_subset = [graphs[i] for i in train_idx]
    test_subset = [graphs[i] for i in test_idx]

    train_loader = DataLoader(train_subset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_subset, batch_size=batch_size, shuffle=False)

    in_dim = int(graphs[0].num_node_features)
    num_classes = int(len(np.unique(labels)))

    model = GINEncoderClassifier(
        in_dim=in_dim,
        hidden_dim=hidden_dim,
        num_layers=num_layers,
        num_classes=num_classes,
        dropout=dropout,
    )

    train_time = train_gin_model(
        model,
        train_loader,
        device=device,
        epochs=epochs,
        lr=lr,
        weight_decay=weight_decay,
    )

    # Embeddings for ALL graphs
    full_loader = DataLoader(graphs, batch_size=batch_size, shuffle=False)
    start = time.time()
    emb = infer_gin_embeddings(model, full_loader, device=device)
    gen_time = time.time() - start

    acc_test = gin_accuracy(model, test_loader, device=device)

    info = {
        "gin_train_time (s)": train_time,
        "gin_gen_time (s)": gen_time,
        "gin_test_acc": acc_test,
        "gin_hidden_dim": hidden_dim,
        "gin_num_layers": num_layers,
        "gin_dropout": dropout,
        "gin_epochs": epochs,
        "gin_batch_size": batch_size,
        "gin_lr": lr,
        "gin_weight_decay": weight_decay,
    }
    return emb, info

def compute_embeddings(method_name: str,
                       graphs_pyg: list,
                       labels: np.ndarray | None = None,
                       embedding_dim: int = 128,
                       seed: int = 42,
                       device: torch.device = DEVICE,
                       gin_params: dict | None = None):
    """Compute embeddings for supported methods: Graph2Vec, NetLSD, GIN."""
    m = method_name.lower()

    # Unsupervised (KarateClub)
    if m in {"graph2vec", "netlsd"}:
        nx_graphs = [pyg_to_nx(g) for g in graphs_pyg]
        start = time.time()
        if m == "graph2vec":
            model = Graph2Vec(dimensions=embedding_dim, seed=seed)
            model.fit(nx_graphs)
            emb = model.get_embedding()
        else:  # netlsd
            model = NetLSD()
            model.fit(nx_graphs)
            emb = model.get_embedding()
        t = time.time() - start
        return emb, {"embedding_time (s)": t}

    # Supervised (PyG)
    if m == "gin":
        if labels is None:
            raise ValueError("GIN requires labels to train.")
        gin_params = gin_params or {}
        # Map embedding_dim -> hidden_dim (we keep naming consistent with saved emb dim)
        hidden_dim = int(gin_params.get("hidden_dim", embedding_dim))
        info_emb, info = compute_embeddings_gin(
            graphs_pyg=graphs_pyg,
            labels=labels,
            hidden_dim=hidden_dim,
            num_layers=int(gin_params.get("num_layers", 5)),
            dropout=float(gin_params.get("dropout", 0.5)),
            epochs=int(gin_params.get("epochs", 80)),
            batch_size=int(gin_params.get("batch_size", 128)),
            lr=float(gin_params.get("lr", 1e-3)),
            weight_decay=float(gin_params.get("weight_decay", 5e-4)),
            seed=seed,
            device=device,
        )
        return info_emb, info

    raise ValueError(f"Unsupported method: {method_name}")


In [80]:
def compute_embedding_change(emb_orig: np.ndarray, emb_pert: np.ndarray):
    """
    Returns mean cosine similarity and mean L2 distance per graph.
    Requires same shape (N, D).
    """
    if emb_orig.shape != emb_pert.shape:
        raise ValueError(
            f"Embedding shapes differ: orig={emb_orig.shape}, pert={emb_pert.shape}. "
            "Make sure recomputed embeddings have the SAME (N, D) as the saved ones."
        )

    a = emb_orig.astype(np.float64, copy=False)
    b = emb_pert.astype(np.float64, copy=False)

    a_norm = a / (np.linalg.norm(a, axis=1, keepdims=True) + 1e-12)
    b_norm = b / (np.linalg.norm(b, axis=1, keepdims=True) + 1e-12)

    cos = np.sum(a_norm * b_norm, axis=1)      # (N,)
    l2  = np.linalg.norm(a - b, axis=1)        # (N,)

    return {
        "mean_cosine": float(np.mean(cos)),
        "std_cosine": float(np.std(cos)),
        "mean_l2": float(np.mean(l2)),
        "std_l2": float(np.std(l2)),
    }

In [81]:
def append_to_stability_log(method_name: str, row: dict):
    log_path = os.path.join(RESULTS_DIR, f"{method_name}_stability.csv")
    df = pd.DataFrame([row])
    if os.path.exists(log_path):
        df.to_csv(log_path, mode="a", header=False, index=False)
    else:
        df.to_csv(log_path, mode="w", header=True, index=False)
    print("Logged:", log_path)

In [96]:
def run_stability_pipeline_enzymes(
    method_name: str,
    dataset: str,
    experiment_num: str,
    perturb_levels=(0.0, 0.05, 0.10, 0.20),
    seeds=(42, 43, 44),
    edge_add_equals_remove: bool = True,
    attr_shuffle: bool = True,
    svm_C: float = 10.0,
    svm_gamma="scale",
):
    ds = load_enzymes_dataset(dataset=dataset)
    #graphs_orig = [ds[i] for i in range(len(ds))]
    keep_indices = get_nonempty_graph_indices(ds)
    graphs_orig = [ds[i] for i in keep_indices]

    # Load original embeddings/labels from classification
    emb_orig, y = load_embeddings_and_labels(method_name, experiment_num, base_dir=CLASS_DIR)

    print("emb_orig rows:", emb_orig.shape[0])
    print("labels rows:", len(y))

    # VALIDATION
    if emb_orig.ndim != 2:
                    raise ValueError(f"eemb_orig must be 2D array (N, D). Got shape: {emb_orig.shape}")
    if len(graphs_orig) != emb_orig.shape[0]:
              raise ValueError(
                f"Mismatch between #graphs and emb_orig. "
                f"#graphs={len(graphs_orig)} but emb_orig.shape[0]={emb_orig.shape[0]}"
              )

    # Lock embedding_dim to saved embeddings (important!)
    embedding_dim = emb_orig.shape[1]
    print(f"[INFO] Using embedding_dim from saved embeddings: {embedding_dim}")

    # Baseline accuracy on original embeddings
    acc_orig = svm_accuracy(emb_orig, y, C=svm_C, gamma=svm_gamma, seed=42)

    for lvl in perturb_levels:
        for seed in seeds:
            remove_pct = float(lvl)
            add_pct = float(lvl) if edge_add_equals_remove else 0.0

            # --- perturb graphs ---
            t0 = time.time()
            graphs_pert = make_perturbed_graphs(
                ds,
                edge_remove_pct=remove_pct,
                edge_add_pct=add_pct,
                attr_shuffle=attr_shuffle,
                seed=seed,
                indices = keep_indices
            )
            perturb_time = time.time() - t0

            # --- recompute embeddings on perturbed graphs ---
            emb_pert, emb_info = compute_embeddings(
                method_name=method_name,
                graphs_pyg=graphs_pert,
                labels=y,
                embedding_dim=embedding_dim,  # locked to emb_orig
                seed=seed,
                device=DEVICE,
                gin_params= {
                    "hidden_dim": embedding_dim,
                    "num_layers": 5,
                    "dropout": 0.5,
                    "epochs": 80,
                    "batch_size": 128,
                    "lr": 1e-3,
                    "weight_decay": 5e-4,
                } if method_name.lower() == "gin" else None
            )

            # VALIDATION
            if emb_pert.ndim != 2:
                    raise ValueError(f"emb_pert must be 2D array (N, D). Got shape: {emb_pert.shape}")
            if len(graphs_pert) != emb_pert.shape[0]:
                  raise ValueError(
                    f"Mismatch between #graphs and emb_pert. "
                    f"#graphs={len(graphs_pert)} but emb_pert.shape[0]={emb_pert.shape[0]}"
                )

            # debug print
            print("orig:", emb_orig.shape, "pert:", emb_pert.shape)

            # stability metrics
            change = compute_embedding_change(emb_orig, emb_pert)

            # accuracy on perturbed embeddings
            acc_pert = svm_accuracy(emb_pert, y, C=svm_C, gamma=svm_gamma, seed=42)
            acc_drop = acc_orig - acc_pert

            row = {
                "experiment_num": experiment_num,
                "dataset": "ENZYMES",
                "embedding_type": method_name,
                "seed": seed,
                "edge_remove_pct": remove_pct,
                "edge_add_pct": add_pct,
                "attr_shuffle": attr_shuffle,
                "perturb_time (s)": perturb_time,
                **emb_info, # embedding info includes timing for karateclub and training/gen info for GIN
                "acc_orig": acc_orig,
                "acc_pert": acc_pert,
                "acc_drop": acc_drop,
                **change
            }

            append_to_stability_log(method_name, row)

    print("Stability run completed.")

In [97]:
run_stability_pipeline_enzymes(
    method_name="GIN",      # Graph2Vec | "NetLSD" | "GIN"
    dataset="MUTAG",
    experiment_num="21112025_1511",
    perturb_levels=(0.0, 0.10),
    seeds=(42,),
    attr_shuffle=True
)

Loaded ENZYMES: 188 graphs | num_classes: 2 | num_node_features: 7
emb_orig rows: 188
labels rows: 188
[INFO] Using embedding_dim from saved embeddings: 64
orig: (188, 64) pert: (188, 64)
Logged: /content/drive/MyDrive/InformationSystems/Stability/results/GIN_stability.csv
orig: (188, 64) pert: (188, 64)
Logged: /content/drive/MyDrive/InformationSystems/Stability/results/GIN_stability.csv
Stability run completed.
